In [1]:
import logging
#from io_process import InputOutputProcessor
#from preprocess import MyVocabularyProcessor
from scipy.spatial.distance import cosine
from logging.handlers import RotatingFileHandler

import numpy as np
import tensorflow as tf

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
reload(sys)
sys.setdefaultencoding("utf-8")

In [3]:
# Parameters
# ==================================================
model_run_number = 0
model_iteration_number = 330000

# Model Parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("max_document_length", 50, "Longest input length (default: 30)")
tf.flags.DEFINE_string("checkpoint_directory", "runs/" + str(model_run_number) + "/checkpoints/", "Checkpoint directory from training run")
tf.flags.DEFINE_string("eval_filepath", "validation.txt0", "Evaluate on this data (Default: None)")
tf.flags.DEFINE_string("vocab_filepath", "runs/" + str(model_run_number) + "/checkpoints/vocab", "Load training time vocabulary (Default: None)")
tf.flags.DEFINE_string("model", "runs/" + str(model_run_number) + "/checkpoints/model-" + str(model_iteration_number), "Load trained model checkpoint (Default: None)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")


FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

if FLAGS.eval_filepath==None or FLAGS.vocab_filepath==None or FLAGS.model==None :
    print("Eval or Vocab filepaths are empty.")
    exit()

# load data and map id-transform based on training time vocabulary
ioProcessor = InputOutputProcessor()

print("\nLoading webapp...\n")

ArgumentError: argument --batch_size: conflicting option string: --batch_size

In [4]:
# Model loading
# ==================================================
checkpoint_file = FLAGS.model
print(checkpoint_file)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        sess.run(tf.initialize_all_variables())
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x1 = graph.get_operation_by_name("input_x1").outputs[0]
        input_x2 = graph.get_operation_by_name("input_x2").outputs[0]
        input_y = graph.get_operation_by_name("input_y").outputs[0]

        dropout_keep_probability = graph.get_operation_by_name("dropout_keep_probability").outputs[0]
        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/distance").outputs[0]

        accuracy = graph.get_operation_by_name("accuracy/accuracy").outputs[0]

        similarity = graph.get_operation_by_name("accuracy/similarity").outputs[0]

        #output_biLSTM_1 = graph.get_operation_by_name("output/backwardleft_1/backwardleft/concat_29").outputs
        #output_biLSTM_2 = graph.get_operation_by_name("output/backwardright_1/backwardright/concat_29").outputs


        output_biLSTM_1 = graph.get_operation_by_name("output/out_branch_left").outputs
        output_biLSTM_2 = graph.get_operation_by_name("output/out_branch_right").outputs

        #emb = graph.get_operation_by_name("embedding/W").outputs[0]
        #embedded_chars = tf.nn.embedding_lookup(emb,input_x)
        # Generate batches for one epoch

f = open('output','w')

NameError: name 'FLAGS' is not defined